# 1) Simulation of Navier-Stokes equation in 2D with pesudo-spectral code

In [ ]:
# credits, inspired from P.Mocz https://github.com/pmocz
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 200  # Set limit to 100 MB

# interactive controls (only in Jupyter)
%matplotlib notebook

# Optimized curl function for visualization
def curl_optimized(vx, vy, kx, ky):
    """return curl of (vx,vy) using FFTs"""
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    omega_hat = 1j * kx * vy_hat - 1j * ky * vx_hat
    return np.real(np.fft.ifftn(omega_hat))

# Simulation parameters
N = 64  # Spatial resolution
t = 0  # current time
tEnd = 5  # end time
dt = 0.001  # timestep
nu = 0.001  # viscosity
tOut = 0.01  # output frequency
plotRealTime = True

# Domain [0,1] x [0,1]
L = 1
xlin = np.linspace(0, L, num=N + 1)[0:N]  # periodic domain
xx, yy = np.meshgrid(xlin, xlin)


# Initial velocity field 
# 1) just a large scale vortex
vx = -np.sin(2 * np.pi * yy)
vy =  np.sin(2 * np.pi * xx * 2)


# Fourier space setup
klin = 2.0 * np.pi / L * np.arange(-N / 2, N / 2)
kx, ky = np.meshgrid(klin, klin)
kx = np.fft.ifftshift(kx) # shift the wave vector so that k=0 comes first 
ky = np.fft.ifftshift(ky)
kSq = kx**2 + ky**2
kSq_inv = np.zeros_like(kSq)
#kSq_inv = np.divide(1.0, kSq, where=kSq > 0)
np.divide(1.0, kSq, out=kSq_inv, where=kSq > 0) # Avoid division by zero

# Precompute diffusion denominator
diff_denom = 1.0 + dt * nu * kSq

# Dealias filter
kmax_dealias = (2.0 / 3.0) * np.max(klin)
dealias = (np.abs(kx) < kmax_dealias) & (np.abs(ky) < kmax_dealias)

# Simulation setup
Nt = int(np.ceil(tEnd / dt))
frameskip = int(tOut / dt)

# Storage for animation
wz_frames = []
vx_frames = []
vy_frames = []

# Main loop (optimized)
for i in range(Nt):
    # 1. Transform velocity to Fourier space
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    
    # 2. Compute gradients in real space
    dvx_x = np.real(np.fft.ifftn(1j * kx * vx_hat))
    dvx_y = np.real(np.fft.ifftn(1j * ky * vx_hat))
    dvy_x = np.real(np.fft.ifftn(1j * kx * vy_hat))
    dvy_y = np.real(np.fft.ifftn(1j * ky * vy_hat))
    
    # 3. Compute nonlinear terms
    rhs_x = -(vx * dvx_x + vy * dvx_y)
    rhs_y = -(vx * dvy_x + vy * dvy_y)
    
    # 4. Transform nonlinear terms to Fourier space and apply dealias
    rhs_x_hat = np.fft.fftn(rhs_x) * dealias
    rhs_y_hat = np.fft.fftn(rhs_y) * dealias
    
    # 5. Build and add forcing (in Fourier space)
    # we do this later
    
    # 6. Compute divergence for pressure solve
    div_rhs_hat = 1j * (kx * rhs_x_hat + ky * rhs_y_hat)
    
    # 7. Solve Poisson equation in Fourier space
    P_hat = -div_rhs_hat * kSq_inv
    
    # 8. Compute pressure gradient in Fourier space
    dPx_hat = 1j * kx * P_hat
    dPy_hat = 1j * ky * P_hat
    
    # 9. Update velocity in Fourier space
    vx_hat = (vx_hat + dt * (rhs_x_hat - dPx_hat)) / diff_denom
    vy_hat = (vy_hat + dt * (rhs_y_hat - dPy_hat)) / diff_denom
    
    # 10. Transform back to real space
    vx = np.real(np.fft.ifftn(vx_hat))
    vy = np.real(np.fft.ifftn(vy_hat))
    
    # Store frame for output
    if i % frameskip == 0 or i == Nt - 1:
        wz = curl_optimized(vx, vy, kx, ky)
        wz_frames.append(wz)
        vx_frames.append(vx.copy())
        vy_frames.append(vy.copy())
    
    t += dt

# Visualization code 
fig, ax = plt.subplots(figsize=(3, 3))
wzmin = np.min(wz_frames)
wzmax = np.max(wz_frames)
im = ax.imshow(wz_frames[0], cmap="RdBu", vmin=wzmin, vmax=wzmax, extent=[0, L, 0, L], origin='lower', aspect='auto')
ax.set_title("Vorticity")
ax.set_xlabel('x')
ax.set_ylabel('y')

def update(frame):
    im.set_data(wz_frames[frame])
    return [im] 

ani = FuncAnimation(fig, update, frames=len(wz_frames), interval=50, blit=True)
HTML(ani.to_jshtml())

# 2) Forced NS equation of steady cellular flow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 200  # Set limit to 100 MB

# interactive controls (only in Jupyter)
%matplotlib notebook

# Optimized curl function for visualization
def curl_optimized(vx, vy, kx, ky):
    """return curl of (vx,vy) using FFTs"""
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    omega_hat = 1j * kx * vy_hat - 1j * ky * vx_hat
    return np.real(np.fft.ifftn(omega_hat))

# Simulation parameters
N = 64  # Spatial resolution
t = 0  # current time
tEnd = 5  # end time
dt = 0.001  # timestep
nu = 0.001  # viscosity
tOut = 0.01  # output frequency
plotRealTime = True

# Domain [0,1] x [0,1]
L = 1
xlin = np.linspace(0, L, num=N + 1)[0:N]  # periodic domain
xx, yy = np.meshgrid(xlin, xlin)

# Initial velocity field 
# 1) just a large scale vortex
# vx = -np.sin(2 * np.pi * yy)
# vy =  np.sin(2 * np.pi * xx * 2)
# cellular flow
vx =  np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
vy = -np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy) 

# Forcing
fA = nu*(2.*np.pi)**2.
fx =  fA*np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
fy = -fA*np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy)

# Fourier space setup
klin = 2.0 * np.pi / L * np.arange(-N / 2, N / 2)
kx, ky = np.meshgrid(klin, klin)
kx = np.fft.ifftshift(kx)
ky = np.fft.ifftshift(ky)
kSq = kx**2 + ky**2
kSq_inv = np.zeros_like(kSq)
#kSq_inv = np.divide(1.0, kSq, where=kSq > 0)
np.divide(1.0, kSq, out=kSq_inv, where=kSq > 0) # Avoid division by zero

# Precompute diffusion denominator
diff_denom = 1.0 + dt * nu * kSq

# Dealias filter
kmax_dealias = (2.0 / 3.0) * np.max(klin)
dealias = (np.abs(kx) < kmax_dealias) & (np.abs(ky) < kmax_dealias)

# Simulation setup
Nt = int(np.ceil(tEnd / dt))
frameskip = int(tOut / dt)

# Storage for animation
wz_frames = []
vx_frames = []
vy_frames = []

# Main loop (optimized)
for i in range(Nt):
    # 1. Transform velocity to Fourier space
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    
    # 2. Compute gradients in real space
    dvx_x = np.real(np.fft.ifftn(1j * kx * vx_hat))
    dvx_y = np.real(np.fft.ifftn(1j * ky * vx_hat))
    dvy_x = np.real(np.fft.ifftn(1j * kx * vy_hat))
    dvy_y = np.real(np.fft.ifftn(1j * ky * vy_hat))
    
    # 3. Compute nonlinear terms
    rhs_x = -(vx * dvx_x + vy * dvx_y)
    rhs_y = -(vx * dvy_x + vy * dvy_y)
    
    # 4. Transform nonlinear terms to Fourier space and apply dealias
    rhs_x_hat = np.fft.fftn(rhs_x) * dealias
    rhs_y_hat = np.fft.fftn(rhs_y) * dealias
    
    # 5. Build and add forcing (in Fourier space)
    fx =  fA*np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
    fy = -fA*np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy)
    fx_hat = np.fft.fftn(fx)
    fy_hat = np.fft.fftn(fy)
    rhs_x_hat += fx_hat
    rhs_y_hat += fy_hat
    
    # 6. Compute divergence for pressure solve
    div_rhs_hat = 1j * (kx * rhs_x_hat + ky * rhs_y_hat)
    
    # 7. Solve Poisson equation in Fourier space
    P_hat = -div_rhs_hat * kSq_inv
    
    # 8. Compute pressure gradient in Fourier space
    dPx_hat = 1j * kx * P_hat
    dPy_hat = 1j * ky * P_hat
    
    # 9. Update velocity in Fourier space
    vx_hat = (vx_hat + dt * (rhs_x_hat - dPx_hat)) / diff_denom
    vy_hat = (vy_hat + dt * (rhs_y_hat - dPy_hat)) / diff_denom
    
    # 10. Transform back to real space
    vx = np.real(np.fft.ifftn(vx_hat))
    vy = np.real(np.fft.ifftn(vy_hat))
    
    # Store frame for output
    if i % frameskip == 0 or i == Nt - 1:
        wz = curl_optimized(vx, vy, kx, ky)
        wz_frames.append(wz)
        vx_frames.append(vx.copy())
        vy_frames.append(vy.copy())
    
    t += dt

# Visualization code 
fig, ax = plt.subplots(figsize=(3, 3))
wzmin = np.min(wz_frames)
wzmax = np.max(wz_frames)
im = ax.imshow(wz_frames[0], cmap="RdBu", vmin=wzmin, vmax=wzmax, extent=[0, L, 0, L], origin='lower', aspect='auto')
stream = ax.streamplot(xx, yy, vx_frames[0], vy_frames[0], density=0.6, color='k', linewidth=1)
ax.set_title("Vorticity")
ax.set_xlabel('x')
ax.set_ylabel('y')

def update(frame):
    im.set_data(wz_frames[frame])
    return [im] 

ani = FuncAnimation(fig, update, frames=len(wz_frames), interval=50, blit=True)
HTML(ani.to_jshtml())

# 3) Steady cellular flow with fluid tracers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 200  # Set limit to 100 MB

# interactive controls (only in Jupyter)
%matplotlib notebook

# Optimized curl function for visualization
def curl_optimized(vx, vy, kx, ky):
    """return curl of (vx,vy) using FFTs"""
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    omega_hat = 1j * kx * vy_hat - 1j * ky * vx_hat
    return np.real(np.fft.ifftn(omega_hat))

def interpolate_vector(vx_field, vy_field, xp, yp, N_grid):
    """
    Interpolate velocity field to particle positions using bilinear interpolation.
    """
    # Normalize particle positions to grid coordinates
    xp_norm = xp * N_grid
    yp_norm = yp * N_grid
    
    # Grid indices (bottom-left corner of the cell containing the particle)
    i0 = np.floor(xp_norm).astype(int) % N_grid
    j0 = np.floor(yp_norm).astype(int) % N_grid
    i1 = (i0 + 1) % N_grid  # next grid index (with periodic wrap)
    j1 = (j0 + 1) % N_grid
    
    # Fractional distances within the cell
    dx = xp_norm - i0
    dy = yp_norm - j0
    
    # Bilinear interpolation weights
    w00 = (1 - dx) * (1 - dy)
    w01 = (1 - dx) * dy
    w10 = dx * (1 - dy)
    w11 = dx * dy
    
    # Interpolate vx and vy components
    vxp = w00 * vx_field[j0, i0] + w01 * vx_field[j0, i1] + \
           w10 * vx_field[j1, i0] + w11 * vx_field[j1, i1]
    vyp = w00 * vy_field[j0, i0] + w01 * vy_field[j0, i1] + \
           w10 * vy_field[j1, i0] + w11 * vy_field[j1, i1]
    
    return vxp, vyp

# Simulation parameters
N = 64  # Spatial resolution
t = 0  # current time
tEnd = 5  # end time
dt = 0.001  # timestep
nu = 0.001  # viscosity
tOut = 0.01  # output frequency
plotRealTime = True

# Domain [0,1] x [0,1]
L = 1
xlin = np.linspace(0, L, num=N + 1)[0:N]  # periodic domain
xx, yy = np.meshgrid(xlin, xlin)

# Initialize particles randomly in the domain
xp = np.random.rand(Np) * L
yp = np.random.rand(Np) * L
vxp = np.zeros(Np)
vyp = np.zeros(Np)

# Initial velocity field 
# 1) just a large scale vortex
# vx = -np.sin(2 * np.pi * yy)
# vy =  np.sin(2 * np.pi * xx * 2)
# cellular flow
vx =  np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
vy = -np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy) 

# Forcing
fA = nu*(2.*np.pi)**2.
fx =  fA*np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
fy = -fA*np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy)

# Fourier space setup
klin = 2.0 * np.pi / L * np.arange(-N / 2, N / 2)
kx, ky = np.meshgrid(klin, klin)
kx = np.fft.ifftshift(kx)
ky = np.fft.ifftshift(ky)
kSq = kx**2 + ky**2
kSq_inv = np.zeros_like(kSq)
#kSq_inv = np.divide(1.0, kSq, where=kSq > 0)
np.divide(1.0, kSq, out=kSq_inv, where=kSq > 0) # Avoid division by zero

# Precompute diffusion denominator
diff_denom = 1.0 + dt * nu * kSq

# Dealias filter
kmax_dealias = (2.0 / 3.0) * np.max(klin)
dealias = (np.abs(kx) < kmax_dealias) & (np.abs(ky) < kmax_dealias)

# Simulation setup
Nt = int(np.ceil(tEnd / dt))
frameskip = int(tOut / dt)

# Storage for animation
wz_frames = []
particle_frames = []  # Stores (xp, yp) for each output frame
vx_frames = []
vy_frames = []

# Main loop (optimized)
for i in range(Nt):
    # 1. Transform velocity to Fourier space
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    
    # 2. Compute gradients in real space
    dvx_x = np.real(np.fft.ifftn(1j * kx * vx_hat))
    dvx_y = np.real(np.fft.ifftn(1j * ky * vx_hat))
    dvy_x = np.real(np.fft.ifftn(1j * kx * vy_hat))
    dvy_y = np.real(np.fft.ifftn(1j * ky * vy_hat))
    
    # 3. Compute nonlinear terms
    rhs_x = -(vx * dvx_x + vy * dvx_y)
    rhs_y = -(vx * dvy_x + vy * dvy_y)
    
    # 4. Transform nonlinear terms to Fourier space and apply dealias
    rhs_x_hat = np.fft.fftn(rhs_x) * dealias
    rhs_y_hat = np.fft.fftn(rhs_y) * dealias
    
    # 5. Build and add forcing (in Fourier space)
    fx =  fA*np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
    fy = -fA*np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy)
    fx_hat = np.fft.fftn(fx)
    fy_hat = np.fft.fftn(fy)
    rhs_x_hat += fx_hat
    rhs_y_hat += fy_hat
    
    # 6. Compute divergence for pressure solve
    div_rhs_hat = 1j * (kx * rhs_x_hat + ky * rhs_y_hat)
    
    # 7. Solve Poisson equation in Fourier space
    P_hat = -div_rhs_hat * kSq_inv
    
    # 8. Compute pressure gradient in Fourier space
    dPx_hat = 1j * kx * P_hat
    dPy_hat = 1j * ky * P_hat
    
    # 9. Update velocity in Fourier space
    vx_hat = (vx_hat + dt * (rhs_x_hat - dPx_hat)) / diff_denom
    vy_hat = (vy_hat + dt * (rhs_y_hat - dPy_hat)) / diff_denom
    
    # 10. Transform back to real space
    vx = np.real(np.fft.ifftn(vx_hat))
    vy = np.real(np.fft.ifftn(vy_hat))

    # Advect particles using the updated velocity field
    vxp, vyp = interpolate_vector(vx, vy, xp, yp, N)

    # second order AB (at time step 0 Euler)
    if i == 0: 
        vxp_old = vxp
        vyp_old = vyp

    xp = (xp + (1.5*vxp - 0.5*vxp_old) * dt) % L
    yp = (yp + (1.5*vyp - 0.5*vyp_old) * dt) % L

    vxp_old = vxp
    vyp_old = vyp
    
    # Store frame for output
    if i % frameskip == 0 or i == Nt - 1:
        wz = curl_optimized(vx, vy, kx, ky)
        wz_frames.append(wz)
        particle_frames.append((xp.copy(), yp.copy()))
        vx_frames.append(vx.copy())
        vy_frames.append(vy.copy())
    
    t += dt

# Visualization code 
fig, ax = plt.subplots(figsize=(3, 3))
wzmin = np.min(wz_frames)
wzmax = np.max(wz_frames)
im = ax.imshow(wz_frames[0], cmap="RdBu", vmin=wzmin, vmax=wzmax,
               extent=[0, L, 0, L], origin='lower', aspect='auto')
scat = ax.scatter(particle_frames[0][0], particle_frames[0][1], s=2.5, c='k', alpha=0.6)
ax.set_title("Vorticity with Lagrangian particles")
ax.set_xlabel('x')
ax.set_ylabel('y')

def update(frame):
    im.set_data(wz_frames[frame])
    scat.set_offsets(np.column_stack(particle_frames[frame]))
    return [im, scat] 

ani = FuncAnimation(fig, update, frames=len(wz_frames), interval=50, blit=True)
HTML(ani.to_jshtml())

# 3) Steady cellular flow with inertial particles (just with Stokes drag)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 200  # Set limit to 100 MB

# interactive controls (only in Jupyter)
%matplotlib notebook

# Optimized curl function for visualization
def curl_optimized(vx, vy, kx, ky):
    """return curl of (vx,vy) using FFTs"""
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    omega_hat = 1j * kx * vy_hat - 1j * ky * vx_hat
    return np.real(np.fft.ifftn(omega_hat))

def interpolate_vector(vx_field, vy_field, xp, yp, N_grid):
    """
    Interpolate velocity field to particle positions using bilinear interpolation.
    """
    # Normalize particle positions to grid coordinates
    xp_norm = xp * N_grid
    yp_norm = yp * N_grid
    
    # Grid indices (bottom-left corner of the cell containing the particle)
    i0 = np.floor(xp_norm).astype(int) % N_grid
    j0 = np.floor(yp_norm).astype(int) % N_grid
    i1 = (i0 + 1) % N_grid  # next grid index (with periodic wrap)
    j1 = (j0 + 1) % N_grid
    
    # Fractional distances within the cell
    dx = xp_norm - i0
    dy = yp_norm - j0
    
    # Bilinear interpolation weights
    w00 = (1 - dx) * (1 - dy)
    w01 = (1 - dx) * dy
    w10 = dx * (1 - dy)
    w11 = dx * dy
    
    # Interpolate vx and vy components
    vxp = w00 * vx_field[j0, i0] + w01 * vx_field[j0, i1] + \
           w10 * vx_field[j1, i0] + w11 * vx_field[j1, i1]
    vyp = w00 * vy_field[j0, i0] + w01 * vy_field[j0, i1] + \
           w10 * vy_field[j1, i0] + w11 * vy_field[j1, i1]
    
    return vxp, vyp

# Simulation parameters
N = 64  # Spatial resolution
t = 0  # current time
tEnd = 5  # end time
dt = 0.001  # timestep
nu = 0.001  # viscosity
tOut = 0.01  # output frequency
plotRealTime = True

# Domain [0,1] x [0,1]
L = 1
xlin = np.linspace(0, L, num=N + 1)[0:N]  # periodic domain
xx, yy = np.meshgrid(xlin, xlin)

# Initialize particles randomly in the domain
xp = np.random.rand(Np) * L
yp = np.random.rand(Np) * L
vxp = np.zeros(Np)
vyp = np.zeros(Np)
taup = np.ones(Np)*0.1

# Initial velocity field 
# 1) just a large scale vortex
# vx = -np.sin(2 * np.pi * yy)
# vy =  np.sin(2 * np.pi * xx * 2)
# cellular flow
vx =  np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
vy = -np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy) 

# Forcing
fA = nu*(2.*np.pi)**2.
fx =  fA*np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
fy = -fA*np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy)

# Fourier space setup
klin = 2.0 * np.pi / L * np.arange(-N / 2, N / 2)
kx, ky = np.meshgrid(klin, klin)
kx = np.fft.ifftshift(kx)
ky = np.fft.ifftshift(ky)
kSq = kx**2 + ky**2
kSq_inv = np.zeros_like(kSq)
#kSq_inv = np.divide(1.0, kSq, where=kSq > 0)
np.divide(1.0, kSq, out=kSq_inv, where=kSq > 0) # Avoid division by zero

# Precompute diffusion denominator
diff_denom = 1.0 + dt * nu * kSq

# Dealias filter
kmax_dealias = (2.0 / 3.0) * np.max(klin)
dealias = (np.abs(kx) < kmax_dealias) & (np.abs(ky) < kmax_dealias)

# Simulation setup
Nt = int(np.ceil(tEnd / dt))
frameskip = int(tOut / dt)

# Storage for animation
wz_frames = []
particle_frames = []  # Stores (xp, yp) for each output frame
vx_frames = []
vy_frames = []

# Main loop (optimized)
for i in range(Nt):
    # 1. Transform velocity to Fourier space
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    
    # 2. Compute gradients in real space
    dvx_x = np.real(np.fft.ifftn(1j * kx * vx_hat))
    dvx_y = np.real(np.fft.ifftn(1j * ky * vx_hat))
    dvy_x = np.real(np.fft.ifftn(1j * kx * vy_hat))
    dvy_y = np.real(np.fft.ifftn(1j * ky * vy_hat))
    
    # 3. Compute nonlinear terms
    rhs_x = -(vx * dvx_x + vy * dvx_y)
    rhs_y = -(vx * dvy_x + vy * dvy_y)
    
    # 4. Transform nonlinear terms to Fourier space and apply dealias
    rhs_x_hat = np.fft.fftn(rhs_x) * dealias
    rhs_y_hat = np.fft.fftn(rhs_y) * dealias
    
    # 5. Build and add forcing (in Fourier space)
    fx =  fA*np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
    fy = -fA*np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy)
    fx_hat = np.fft.fftn(fx)
    fy_hat = np.fft.fftn(fy)
    rhs_x_hat += fx_hat
    rhs_y_hat += fy_hat
    
    # 6. Compute divergence for pressure solve
    div_rhs_hat = 1j * (kx * rhs_x_hat + ky * rhs_y_hat)
    
    # 7. Solve Poisson equation in Fourier space
    P_hat = -div_rhs_hat * kSq_inv
    
    # 8. Compute pressure gradient in Fourier space
    dPx_hat = 1j * kx * P_hat
    dPy_hat = 1j * ky * P_hat
    
    # 9. Update velocity in Fourier space
    vx_hat = (vx_hat + dt * (rhs_x_hat - dPx_hat)) / diff_denom
    vy_hat = (vy_hat + dt * (rhs_y_hat - dPy_hat)) / diff_denom
    
    # 10. Transform back to real space
    vx = np.real(np.fft.ifftn(vx_hat))
    vy = np.real(np.fft.ifftn(vy_hat))

    # Advect particles using the updated velocity field
    vxf, vyf = interpolate_vector(vx, vy, xp, yp, N)
    rhs_vxp = (vxf - vxp) / taup
    rhs_vyp = (vyf - vyp) / taup
    
    if i == 0: 
        rhs_vxp_old = rhs_vxp
        rhs_vyp_old = rhs_vyp
        vxp_old = vxp
        vyp_old = vyp
        
    vxp = vxp + (1.5*rhs_vxp - 0.5*rhs_vxp_old) * dt
    vyp = vyp + (1.5*rhs_vyp - 0.5*rhs_vyp_old) * dt      
    vxp = np.where(taup == 0, vxf, vxp)
    vyp = np.where(taup == 0, vyf, vyp)
    xp = (xp + (1.5*vxp - 0.5*vxp_old) * dt) % L
    yp = (yp + (1.5*vyp - 0.5*vyp_old) * dt) % L
    
    rhs_vxp_old = rhs_vxp
    rhs_vyp_old = rhs_vyp
    vxp_old = vxp
    vyp_old = vyp
    
    # Store frame for output
    if i % frameskip == 0 or i == Nt - 1:
        wz = curl_optimized(vx, vy, kx, ky)
        wz_frames.append(wz)
        particle_frames.append((xp.copy(), yp.copy()))
        vx_frames.append(vx.copy())
        vy_frames.append(vy.copy())
    
    t += dt

# Visualization code 
fig, ax = plt.subplots(figsize=(3, 3))
wzmin = np.min(wz_frames)
wzmax = np.max(wz_frames)
im = ax.imshow(wz_frames[0], cmap="RdBu", vmin=wzmin, vmax=wzmax,
               extent=[0, L, 0, L], origin='lower', aspect='auto')
scat = ax.scatter(particle_frames[0][0], particle_frames[0][1], s=2.5, c='k', alpha=0.6)
ax.set_title("Vorticity with Lagrangian particles")
ax.set_xlabel('x')
ax.set_ylabel('y')

def update(frame):
    im.set_data(wz_frames[frame])
    scat.set_offsets(np.column_stack(particle_frames[frame]))
    return [im, scat] 

ani = FuncAnimation(fig, update, frames=len(wz_frames), interval=50, blit=True)
HTML(ani.to_jshtml())

 # 4) Steady cellular flow with inertial particles with Stokes drag and added-mass

 This is left as exercise. You can use the following parts of code. Verify that heavy particles are ejected, while light particles are injected. And what happens for neutrally buoyant ones? 

In [ ]:
beta = np.ones(Np)*1.5  # or any other value between 0 and 3

    # Added mass term
    Dtvx = np.real(np.fft.ifftn(- dPx_hat - nu*kSq*vx_hat + fx_hat))    
    Dtvy = np.real(np.fft.ifftn(- dPy_hat - nu*kSq*vy_hat + fy_hat)) 
    Dtvxf, Dtvyf = interpolate_vector(Dtvx, Dtvy, xp, yp, N)
    rhs_vxp += beta*Dtvxf
    rhs_vyp += beta*Dtvyf

# 5) Add finite-size Faxen corrections 

You can use the Gaussian convolution approach with FFT presented in Acceleration statistics of  finite-sized particles in turbulent flow: the role of Faxen forces, E. Calzavarini, R. Volk, M. Bourgoin, E. Leveque, J.-F. Pinton and F. Toschi,  J.Fluid Mech., 630, 179-189 (2009). [PDF](http://www.ecalzavarini.info/wp-content/uploads/2022/01/calza_jfm_faxen_09.pdf)

# 6) Implement a random cellular flow via a Ornstein–Uhlenbeck process 

The phases $\theta$ in the sinusoidal forcing terms (4 in our case) follow independent O-U random processes:
$$ d\theta = - \frac{\theta}{\tau_{\theta}} dt + \sqrt{ \frac{2 D_{\theta}}{\tau_{\theta}} dt}\ \eta(t) $$


Try it with different initial conditions : first with zero inital velocity then with the already developed steady cellular flow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 200  # Set limit to 100 MB

# interactive controls (only in Jupyter)
%matplotlib notebook

# Optimized curl function for visualization
def curl_optimized(vx, vy, kx, ky):
    """return curl of (vx,vy) using FFTs"""
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    omega_hat = 1j * kx * vy_hat - 1j * ky * vx_hat
    return np.real(np.fft.ifftn(omega_hat))

def interpolate_vector(vx_field, vy_field, xp, yp, N_grid):
    """
    Interpolate velocity field to particle positions using bilinear interpolation.
    """
    # Normalize particle positions to grid coordinates
    xp_norm = xp * N_grid
    yp_norm = yp * N_grid
    
    # Grid indices (bottom-left corner of the cell containing the particle)
    i0 = np.floor(xp_norm).astype(int) % N_grid
    j0 = np.floor(yp_norm).astype(int) % N_grid
    i1 = (i0 + 1) % N_grid  # next grid index (with periodic wrap)
    j1 = (j0 + 1) % N_grid
    
    # Fractional distances within the cell
    dx = xp_norm - i0
    dy = yp_norm - j0
    
    # Bilinear interpolation weights
    w00 = (1 - dx) * (1 - dy)
    w01 = (1 - dx) * dy
    w10 = dx * (1 - dy)
    w11 = dx * dy
    
    # Interpolate vx and vy components
    vxp = w00 * vx_field[j0, i0] + w01 * vx_field[j0, i1] + \
           w10 * vx_field[j1, i0] + w11 * vx_field[j1, i1]
    vyp = w00 * vy_field[j0, i0] + w01 * vy_field[j0, i1] + \
           w10 * vy_field[j1, i0] + w11 * vy_field[j1, i1]
    
    return vxp, vyp

# Simulation parameters
N = 64  # Spatial resolution
t = 0  # current time
tEnd = 10  # end time
dt = 0.001  # timestep
nu = 0.001  # viscosity
tOut = 0.01  # output frequency
plotRealTime = True

# Domain [0,1] x [0,1]
L = 1
xlin = np.linspace(0, L, num=N + 1)[0:N]  # periodic domain
xx, yy = np.meshgrid(xlin, xlin)

# Initialize particles randomly in the domain
xp = np.random.rand(Np) * L
yp = np.random.rand(Np) * L
vxp = np.zeros(Np)
vyp = np.zeros(Np)
taup = np.ones(Np)*0.1
beta = np.ones(Np)*0.5  # or any other value between 0 and 3

# Initial velocity field 
# 1) just a large scale vortex
# vx = -np.sin(2 * np.pi * yy)
# vy =  np.sin(2 * np.pi * xx * 2)
# cellular flow
vx =  np.cos(2.*np.pi * xx)*np.sin(2.*np.pi * yy)
vy = -np.sin(2.*np.pi * xx)*np.cos(2.*np.pi * yy) 
# zero initial velocity
vx = 0
vy = 0

# Forcing
fA = 10.*nu*(2.*np.pi)**2.
fx =  0.*xx
fy =  0.*xx
phase = np.random.rand(4)*2 * np.pi # 4 random phases
tau_relax = 1.0
diffusion = 10.0
wavenumb = 1
sigma = np.sqrt(2 * diffusion / tau_relax)

# Fourier space setup
klin = 2.0 * np.pi / L * np.arange(-N / 2, N / 2)
kx, ky = np.meshgrid(klin, klin)
kx = np.fft.ifftshift(kx)
ky = np.fft.ifftshift(ky)
kSq = kx**2 + ky**2
kSq_inv = np.zeros_like(kSq)
#kSq_inv = np.divide(1.0, kSq, where=kSq > 0)
np.divide(1.0, kSq, out=kSq_inv, where=kSq > 0) # Avoid division by zero

# Precompute diffusion denominator
diff_denom = 1.0 + dt * nu * kSq

# Dealias filter
kmax_dealias = (2.0 / 3.0) * np.max(klin)
dealias = (np.abs(kx) < kmax_dealias) & (np.abs(ky) < kmax_dealias)

# Simulation setup
Nt = int(np.ceil(tEnd / dt))
frameskip = int(tOut / dt)

# Storage for animation
wz_frames = []
particle_frames = []  # Stores (xp, yp) for each output frame
vx_frames = []
vy_frames = []

# Main loop (optimized)
for i in range(Nt):
    # 1. Transform velocity to Fourier space
    vx_hat = np.fft.fftn(vx)
    vy_hat = np.fft.fftn(vy)
    
    # 2. Compute gradients in real space
    dvx_x = np.real(np.fft.ifftn(1j * kx * vx_hat))
    dvx_y = np.real(np.fft.ifftn(1j * ky * vx_hat))
    dvy_x = np.real(np.fft.ifftn(1j * kx * vy_hat))
    dvy_y = np.real(np.fft.ifftn(1j * ky * vy_hat))
    
    # 3. Compute nonlinear terms
    rhs_x = -(vx * dvx_x + vy * dvx_y)
    rhs_y = -(vx * dvy_x + vy * dvy_y)
    
    # 4. Transform nonlinear terms to Fourier space and apply dealias
    rhs_x_hat = np.fft.fftn(rhs_x) * dealias
    rhs_y_hat = np.fft.fftn(rhs_y) * dealias
    
    # 5. Build and add forcing (in Fourier space)
    # OU process for random phases
    dphi = - phase * dt / tau_relax + sigma * np.sqrt(dt) * np.random.randn(4)
    phase = (phase + dphi) % (2 * np.pi) 
    fx =  fA*np.cos(wavenumb*2.*np.pi * xx + phase[0])*np.sin(wavenumb*2.*np.pi * yy + phase[1])
    fy = -fA*np.sin(wavenumb*2.*np.pi * xx + phase[2])*np.cos(wavenumb*2.*np.pi * yy + phase[3])
    fx_hat = np.fft.fftn(fx)
    fy_hat = np.fft.fftn(fy)
    rhs_x_hat += fx_hat
    rhs_y_hat += fy_hat
    
    # 6. Compute divergence for pressure solve
    div_rhs_hat = 1j * (kx * rhs_x_hat + ky * rhs_y_hat)
    
    # 7. Solve Poisson equation in Fourier space
    P_hat = -div_rhs_hat * kSq_inv
    
    # 8. Compute pressure gradient in Fourier space
    dPx_hat = 1j * kx * P_hat
    dPy_hat = 1j * ky * P_hat
    
    # 9. Update velocity in Fourier space
    vx_hat = (vx_hat + dt * (rhs_x_hat - dPx_hat)) / diff_denom
    vy_hat = (vy_hat + dt * (rhs_y_hat - dPy_hat)) / diff_denom
    
    # 10. Transform back to real space
    vx = np.real(np.fft.ifftn(vx_hat))
    vy = np.real(np.fft.ifftn(vy_hat))

    # Advect particles using the updated velocity field
    vxf, vyf = interpolate_vector(vx, vy, xp, yp, N)
    rhs_vxp = (vxf - vxp) / taup
    rhs_vyp = (vyf - vyp) / taup

    # Added mass term
    Dtvx = np.real(np.fft.ifftn(- dPx_hat - nu*kSq*vx_hat + fx_hat))    
    Dtvy = np.real(np.fft.ifftn(- dPy_hat - nu*kSq*vy_hat + fy_hat)) 
    Dtvxf, Dtvyf = interpolate_vector(Dtvx, Dtvy, xp, yp, N)
    rhs_vxp += beta*Dtvxf
    rhs_vyp += beta*Dtvyf
    
    if i == 0: 
        rhs_vxp_old = rhs_vxp
        rhs_vyp_old = rhs_vyp
        vxp_old = vxp
        vyp_old = vyp
        
    vxp = vxp + (1.5*rhs_vxp - 0.5*rhs_vxp_old) * dt
    vyp = vyp + (1.5*rhs_vyp - 0.5*rhs_vyp_old) * dt      
    vxp = np.where(taup == 0, vxf, vxp)
    vyp = np.where(taup == 0, vyf, vyp)
    xp = (xp + (1.5*vxp - 0.5*vxp_old) * dt) % L
    yp = (yp + (1.5*vyp - 0.5*vyp_old) * dt) % L
    
    rhs_vxp_old = rhs_vxp
    rhs_vyp_old = rhs_vyp
    vxp_old = vxp
    vyp_old = vyp
    
    # Store frame for output
    if i % frameskip == 0 or i == Nt - 1:
        wz = curl_optimized(vx, vy, kx, ky)
        wz_frames.append(wz)
        particle_frames.append((xp.copy(), yp.copy()))
        vx_frames.append(vx.copy())
        vy_frames.append(vy.copy())
    
    t += dt

# Visualization code 
fig, ax = plt.subplots(figsize=(3, 3))
wzmin = np.min(wz_frames)
wzmax = np.max(wz_frames)
im = ax.imshow(wz_frames[0], cmap="RdBu", vmin=wzmin, vmax=wzmax,
               extent=[0, L, 0, L], origin='lower', aspect='auto')
scat = ax.scatter(particle_frames[0][0], particle_frames[0][1], s=2.5, c='k', alpha=0.6)
ax.set_title("Vorticity with Lagrangian particles")
ax.set_xlabel('x')
ax.set_ylabel('y')

def update(frame):
    im.set_data(wz_frames[frame])
    scat.set_offsets(np.column_stack(particle_frames[frame]))
    return [im, scat] 

ani = FuncAnimation(fig, update, frames=len(wz_frames), interval=50, blit=True)
HTML(ani.to_jshtml())